--- 
authors: Freek Pols
updated: december 3, 2025    
---
# Druksensor ijken en maken van een $pV$-diagram

## Introductie

In de experimentele natuurkunde was het lang geleden gelukt om de krachten tussen ladingen te bestuderen zonder dat bekend was hoe groot die ladingen nu precies waren. Men laadde een metalen bol op en hield deze tegen een andere metalen bol van hetzelfde materiaal. Men redeneerde dat de ladingen op de bollen gelijk waren, omdat ze van hetzelfde materiaal waren. Vervolgens plaatste men de bollen in een vacuüm en mat men de krachten tussen de bollen met een zeer gevoelige balans. Op deze manier kon men de krachten tussen de ladingen bestuderen zonder de absolute waarde van de ladingen te kennen. Dit trucje kon herhaald worden met andere bollen waarna een kwantiatieve beschrijving van de krachten tussen ladingen mogelijk werd.

Een soortgelijke meettechniek gaan we gebruiken om een druksensor te ijken. Van de sensor zijn wel wat dingen bekend, maar omdat de spanning van de Arduino niet overeenkomstig is met de gewenste spanning, zouden we deze moeten ijken. We weten dat de sensor lineair is, dus als we twee punten weten, kunnen we de rest van de curve bepalen. Nog beter zou het zijn om drie punten te nemen en zo ook het lineaire karakter van de sensor te bevestigen.

## Theorie

Een injectiespuit met een maximaal volume van 50 mL is gevuld met lucht. De spuit kan aan een kant afgesloten worden met een tube die verbonden is met een druksensor die de gasdruk meet. Door de zuiger van de spuit in te drukken, wordt het volume verkleind en de druk verhoogd. Wanneer we de druk langzaam in drukken verwachten we dat de druk in de spuit volgens de wet van Boyle toeneemt:

$$
    P_1 V_1 = P_2 V_2 
$$ (eq:Boyle)

Omdat de gemeten spanning van de druksensor lineair afhankelijk is van de druk, kan de druk uitgedrukt worden als:

$$
    P = a U + b
$$ (eq:lineair)



## Methode en materialen

```{note} Software
De Arduino code staat al op de Arduino's. Als je de Arduino aansluit op je computer en de Arduino IDE opent, kan je de seriële monitor openen om de gemeten spanning te zien.
```

Je maakt gebruik van een Arduino. Daarvoor heb je de juiste IDE nodig. Het programma staat al op de Arduino's in het lokaal. Zodra je de Arduino aansluit op je computer zal de Arduino gaan meten, maar zijn de metingen nog niet zichtbaar. Je moet de Arduino op `Arduino MKR Zero` zetten. Dan wordt nog wel een driver geinstalleerd. 

Controleer of de Arduino herkend wordt door op `tools` -> `port` te klikken, daar staat de com poort van de Arduino. Open vervolgens de seriële monitor (het vergrootglas rechtsboven in de IDE) om de gemeten spanning te zien.

```{warning}
De twee stekkertjes hoef je NIET met elkaar te verbinden. Dit is alleen voor een meting in de brandblusser.
``` 

```{code} C++
int drukpin = A1;

void setup() {
  pinMode(A1,INPUT);
  Serial.begin(9600);
}

void loop() {
  Serial.println(analogRead(drukpin));
  delay(100);
}
```

### Deel 1
Stel de injectiespuit in op 40 mL en sluit de spuit aan op de druksensor door middel van een zo klein mogelijke tube. Meet de spanning van de druksensor met de Arduino en noteer deze waarde als $U_1$. Druk vervolgens de zuiger langzaam in tot 20 mL en meet opnieuw de spanning van de druksensor, noteer deze waarde als $U_2$. Herhaal dit voor volumes van 10 mL. 

1. Leg uit waarom een zo klein mogelijke tube gebruikt moet worden.
2. Welke waarde hoort bij de gasdruk bij 40 mL? Zoek deze waarde op.
3. Welke waarden horen bij de gasdruk bij 20 en 10 mL? 
4. Gebruik de drie punten om de waarden van $a$ en $b$ in {numref}`vergelijking {number} <eq:lineair>` te bepalen en controleer of de sensor inderdaad lineair is door de waarden te plotten.

### Deel 2
Vervang daarbij de kleine tube voor een langere en bepaal het onbekende volume van de tube met een volgende meetserie waarbij je de druk en het volume bepaald. Zorg ervoor dat ook drukken onder de 1 atm gemeten worden. 
```{tip}
Maak gebruik van een systematische fout in het volume om het volume van de tube te vinden.
```

## Resultaten

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

#deel 1
#1 tube moet zo klein mogelijk zijn want daar zit ook lucht in, dus hoe korter de tube is hoe nauwkeuriger zijn de aangegeven maatstrepen op de spuit
#2 dit is de atmosferische druk, dus 1 atm
#3 van 30ml 1.35 bar en van 20ml 2.03 bar, het was bij deel een niet gelukt om 10ml te halen zonder dat de tube eraf vloog

V = [40,30,20]
U = np.array([180, 227, 316], dtype=float)   #measured voltages from arduino
P = np.array([1.00, 1.35, 2.03], dtype=float)   #calculated with p_1 * V_1 = P_2 * v_2

# function for the fit
def functie(U, a, b):
    return a * U + b

# Curve fit
val, cov = curve_fit(functie, U, P)
a, b = val

print("a =", a)
print("b =", b)

U_fit = np.linspace(min(U), max(U), 200)
P_fit = functie(U_fit, a, b)

# Plot
plt.figure()
plt.scatter(U, P, color='red',label='Measured data')
plt.plot(U_fit, P_fit,'b--', label=f'Fit: P = {a:.5f} U + {b:.5f}', linewidth=2)
plt.title("Arduino ijken")
plt.xlabel("Voltage U ")
plt.ylabel("Pressure P (bar)")
plt.savefig("figures/ijken.png", dpi=450)
plt.legend()
plt.show()

#deel 2
#lange_buis
#40 ml+lange_buis = 181 #standaard luchtdruk
#50 ml+lange_buis = 156
#30 ml+lange_buis = 225
#20 ml+lange_buis =299
#10 ml+lange_buis = 460

V = np.array([40, 50, 30, 20, 10], dtype=float)   # syringe volumes in mL
U = np.array([181, 156, 225, 299, 460], dtype=float)    #measured voltages from arduino

#use found values of a and b to get the pressure
P = a * U + b 

# Boyle-law-based model:
# P = Constante / (V + L)
def function(V, Constante, L):
    return Constante / (V + L)

# Fit to find tube volume L
vals, covs = curve_fit(function, V, P, p0=[50, 5])  
Constante, L = vals

print("Constante =", Constante, 'bar·mL')
print("volume lange buis, L =", L, "mL")

# Plot fit
V_fit = np.linspace(min(V), max(V), 200)
P_fit = function(V_fit, Constante, L)

plt.figure(figsize=(7,5))
plt.scatter(V, P, color='red', label='Measured pressures')
plt.plot(V_fit, P_fit,'b--', label=f'Fit: P = C/(V + L), L={L:.2f} mL', linewidth=2)
plt.xlabel("Syringe Volume (mL)")
plt.ylabel("Pressure (bar)")
plt.title("Druk met lange buis")
plt.savefig("figures/lange_buis.png", dpi=450)
plt.legend()
plt.show



```{figure} figures/ijken.png
:width: 70%
:label: fig_ijken 

ijklijn om de voltages van de arduino om te zetten in druk.
```
```{figure} figures/lange_buis.png
:width: 70%
:label: fig_lange_buis 

Fitlijn door de meetpunten met een lange tube, hiermee is het volume van de lange buis bepaald
```